In [ ]:
import pandas as pd 
from sklearn.linear_model import RidgeCV, LinearRegression, Ridge, Lasso, LassoLarsCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
from sklearn.metrics import mean_absolute_error

In [ ]:
raw_data = pd.read_csv('../data/output/mannheim_transformed.csv')

In [ ]:
raw_data.info()

In [ ]:
col_to_drop = ['bike_number','start_position','end_time','end_position','end_position_name']
prediction_data = raw_data.drop(columns=col_to_drop)

In [ ]:
prediction_data

In [ ]:
prediction_data['start_time'] = pd.to_datetime(prediction_data.start_time)

In [ ]:
#from scipy import stats
#z = np.abs(stats.zscore(prediction_data['duration']))
#prediction_data = prediction_data[(z < 3)]

In [ ]:
Q1 = prediction_data['duration'].quantile(0.25)
Q3 = prediction_data['duration'].quantile(0.75)
IQR = Q3 - Q1

In [ ]:
mask = prediction_data['duration'].between((Q1 - 1.5 * IQR), (Q3 + 1.5 * IQR), inclusive=True)
iqr = prediction_data.loc[mask]

In [ ]:
iqr

In [ ]:
prediction_data = iqr.copy()

In [ ]:
# Creating an individual column for hour of the day
prediction_data['HOUR'] = prediction_data.start_time.dt.strftime('%-H').astype('int')

# Creating an individual column for week of the year
prediction_data['WEEK_OF_YEAR'] = prediction_data.start_time.dt.strftime('%W').astype('int')

# Creating an individual column for day of the week
prediction_data['DAY_OF_WEEK'] = prediction_data.start_time.dt.strftime('%w').astype('int')

seasons = []
for month in prediction_data.start_time.dt.strftime('%m').astype('int'):
    if month in [1, 2, 12]:
        seasons.append('WINTER')
    elif month in [3, 4, 5]:
        seasons.append('SPRING')
    elif month in [6, 7, 8]:
        seasons.append('SUMMER')
    elif month in [9, 10, 11]:
        seasons.append('FALL')
prediction_data['season'] = seasons

In [ ]:
# Creating fig and subplots
#fig, axes = plt.subplots(2, 5, figsize=(20, 7), sharex=True, sharey=True, dpi=500)

# Plotting the relationship between the actual count and its lagged values
#for i, ax in enumerate(axes.flatten()[:10]):
#    pd.plotting.lag_plot(raw_data.sort_values(by=['start_time'])['duration'], lag=i + 1, ax=ax)
#    ax.set_title('Lag ' + str(i + 1))
#plt.tight_layout();

In [ ]:
sns.boxplot(x='weekend', y='duration', data=prediction_data)

In [ ]:
sns.boxplot(x='is_station', y='duration', data=prediction_data)

In [ ]:
sns.boxplot(x='season', y='duration', data=prediction_data)

In [ ]:
sns.boxplot(x='HOUR', y='duration', data=prediction_data)

In [ ]:
sns.boxplot(x='WEEK_OF_YEAR', y='duration', data=prediction_data)

In [ ]:
sns.boxplot(x='DAY_OF_WEEK', y='duration', data=prediction_data)

In [ ]:
# Applying sine,cosine transformation on column hour to retain the cyclical nature
prediction_data['HOUR_SIN'] = np.sin(prediction_data.HOUR * (2. * np.pi / 24))
prediction_data['HOUR_COS'] = np.cos(prediction_data.HOUR * (2. * np.pi / 24))

In [ ]:
# Applying sine,cosine transformation on column WEEK_OF_YEAR to retain the cyclical nature
prediction_data['WEEK_OF_YEAR_SIN'] = np.sin(prediction_data.WEEK_OF_YEAR * (2. * np.pi / 52))
prediction_data['WEEK_OF_YEAR_COS'] = np.cos(prediction_data.WEEK_OF_YEAR * (2. * np.pi / 52))

In [ ]:
# Applying sine,cosine transformation on column DAY_OF_WEEK to retain the cyclical nature
prediction_data['DAY_OF_WEEK_SIN'] = np.sin(prediction_data.DAY_OF_WEEK * (2. * np.pi / 7))
prediction_data['DAY_OF_WEEK_COS'] = np.cos(prediction_data.DAY_OF_WEEK * (2. * np.pi / 7))

In [ ]:
seasonal_dummies = pd.get_dummies(prediction_data['season'])

In [ ]:
# Dropping individual time columns since their transformation will be used
prediction_data.drop(columns=['WEEK_OF_YEAR', 'DAY_OF_WEEK', 'HOUR','start_time','season','start_position_name'], axis=1, inplace=True)

In [ ]:
prediction_data = pd.concat([prediction_data, seasonal_dummies], axis=1)

In [ ]:
X = prediction_data.drop(columns=['duration'])
y = prediction_data['duration'].values.reshape(-1,1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [ ]:
target_transformer = StandardScaler().fit(y_train)
y_train = target_transformer.transform(y_train)

In [ ]:
model = LassoLarsCV(cv=5, normalize=False).fit(X_train, y_train)

In [ ]:
y_pred = target_transformer.inverse_transform(model.predict(X_train).reshape(-1, 1))

In [ ]:
reg_residuals = target_transformer.inverse_transform(y_train.reshape(-1, 1)) - y_pred

In [ ]:
# Creating fit and subplots
fix, axs = plt.subplots(2, 1, figsize=(17, 15), sharex=True)

# Plotting actuals and fit of linear and random forest regression
pd.DataFrame({'ACTUALS': target_transformer.inverse_transform(y_train.reshape(-1, 1)).reshape(-1),
              'LinearRegression': y_pred.reshape(-1)}).plot(ax=axs[0])


# Plotting residuals of linear and random forest regression
pd.DataFrame({'LinearRegression': reg_residuals.reshape(-1)}).plot(ax=axs[1])

axs[1].set_title('LinearRegression Residuals');

In [ ]:
mean_absolute_error(target_transformer.inverse_transform(y_train), y_pred)

In [ ]:
rf_mode_opt = RandomForestRegressor(n_jobs=-1, 
                                 random_state=123,
                                 n_estimators=800,
                                 min_samples_split=10,
                                 min_samples_leaf=4,
                                 max_features='sqrt',
                                 max_depth=80,
                                 bootstrap=True)

In [ ]:
rf_mode_opt.fit(X_train, y_train)

In [ ]:
rf_model = RandomForestRegressor(n_jobs=-1, 
                                 random_state=123)

In [ ]:
rf_model.fit(X_train, y_train)

In [ ]:
# Calculating the fit of the model
rf_predictions = target_transformer.inverse_transform(rf_model.predict(X_train).reshape(-1, 1))



# Calculating the residuals
rf_residuals = target_transformer.inverse_transform(y_train.reshape(-1, 1)) - rf_predictions

In [ ]:
# Creating fit and subplots
fix, axs = plt.subplots(2, 1, figsize=(17, 15), sharex=True)

# Plotting actuals and fit of linear and random forest regression
pd.DataFrame({'ACTUALS': target_transformer.inverse_transform(y_train.reshape(-1, 1)).reshape(-1),
              'RandomForestRegression': rf_predictions.reshape(-1)}).plot(ax=axs[0])


# Plotting residuals of linear and random forest regression
pd.DataFrame({'RandomForestRegression': rf_residuals.reshape(-1)}).plot(ax=axs[1])

axs[1].set_title('RandomForest Residuals');

In [ ]:
mean_absolute_error(target_transformer.inverse_transform(y_train.reshape(-1, 1)), rf_predictions)

In [ ]:
# Creating fig
fig = plt.figure(figsize=(20, 10))

# Deriving the feature importances of the optimized and default model
default_importances = list(rf_model.feature_importances_)
optimized_importances = list(rf_mode_opt.feature_importances_)

# Creating an array with range of number of variables
x_values = np.arange(len(default_importances))

# Creating bar plots of optimized and default importances
plt.bar(x_values, default_importances,orientation='vertical', color='blue', width=.5, align='center', label='Default')
plt.bar(x_values + 0.5, optimized_importances, orientation='vertical', color='red', width=.5, align='center', label='Optimized')

plt.xticks(x_values + 0.25, list(X.columns), rotation='vertical')

plt.ylabel('Importance')
plt.xlabel('Variable')
plt.title('Variable Importances')

plt.legend(loc=1)
plt.show();

In [ ]:
#from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
#n_estimators = [int(x) for x in np.linspace(start = 200, stop = 1000, num = 5)]
# Number of features to consider at every split
#max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
#max_depth = [int(x) for x in np.linspace(20, 90, num = 8)]
#max_depth.append(None)
# Minimum number of samples required to split a node
#min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
#min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
#bootstrap = [True, False]
# Create the random grid
#random_grid = {'n_estimators': n_estimators,
#               'max_features': max_features,
#               'max_depth': max_depth,
#               'min_samples_split': min_samples_split,
#               'min_samples_leaf': min_samples_leaf,
#               'bootstrap': bootstrap}
#print(random_grid)

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
#rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
#rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=1, random_state=42, n_jobs = -1)
# Fit the random search model
#rf_random.fit(X_train, y_train)

In [ ]:
#rf_random.best_params_